In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.preprocessing import label_binarize
from itertools import cycle
from sklearn.preprocessing import normalize
from scipy import interp
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.pipeline import Pipeline
sys.path.append('/home/lolo/Desktop/dnnComm/vde/vde/')
from vde import VDE 



name= '/home/lolo/Desktop/dnnComm/RML2016.10a_dict.pkl'
data = pd.read_pickle(name)

In [2]:
list = []
for d in data:
    list.append(d)    

In [3]:
# what is the best normalization strategy?
# time-lag is considered in features, data augmentation and cost funstion.
lag_time = 8
trajs = []
stride = 20
for l in range(len(list)):
    d = data[list[l]]
    for i in range(0, len(d), stride):
        dd = normalize(d[i, :, :], axis=1, norm='l2').T
        #dd = normalize(dd[i, : :], axis=0, norm='l2')
        #dd = d[i, :, :].T
        Dt = dd.shape
        dddd = dd
        for t in range(1, lag_time):
            ddd = np.zeros(Dt)
            ddd[:-t] = dd[:-t] - dd[t:] # lag-differences as features
            dddd = np.append(dddd, ddd, axis=1)
        trajs.append(dddd)
print ('samples,features', trajs[0].shape)    
print ('#data', len(trajs))

samples,features (128, 16)
#data 11000


In [5]:
mdl = VDE(trajs[0].shape[-1], lag_time=lag_time,
          hidden_size=256, hidden_layer_depth=3,
          batch_size=100, n_epochs=1, cuda=False, 
          sliding_window=False, dropout_rate=0.3,
          learning_rate=1E-3, autocorr=True)

In [7]:
# for some reasons autocorr_loss does behave properly!
mdl.fit(trajs)

Epoch: 0
Batch 100, loss = 1.1170
rec_loss = 0.1400, autocorr_loss = 0.9770
Batch 200, loss = 1.0547
rec_loss = 0.1003, autocorr_loss = 0.9544
Batch 300, loss = 0.9983
rec_loss = 0.0775, autocorr_loss = 0.9208
Batch 400, loss = 1.0175
rec_loss = 0.0507, autocorr_loss = 0.9667
Batch 500, loss = 0.8834
rec_loss = 0.0610, autocorr_loss = 0.8224
Batch 600, loss = 0.6471
rec_loss = 0.0288, autocorr_loss = 0.6183
Batch 700, loss = 0.7128
rec_loss = 0.0305, autocorr_loss = 0.6823
Batch 800, loss = 0.6670
rec_loss = 0.0241, autocorr_loss = 0.6430
Batch 900, loss = 0.8033
rec_loss = 0.0248, autocorr_loss = 0.7785
Batch 1000, loss = 0.8740
rec_loss = 0.0184, autocorr_loss = 0.8556
Batch 1100, loss = 0.7816
rec_loss = 0.0181, autocorr_loss = 0.7635
Batch 1200, loss = 0.8548
rec_loss = 0.0186, autocorr_loss = 0.8363
Batch 1300, loss = 0.7524
rec_loss = 0.0153, autocorr_loss = 0.7371
Batch 1400, loss = 0.8712
rec_loss = 0.0132, autocorr_loss = 0.8580
Batch 1500, loss = 0.7072
rec_loss = 0.0171, aut